# Kaggle: Dogs Vs Cats (using VGG16 pretrained network)

In [3]:
import os,shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline

In [15]:
from keras.models import Sequential
from keras.layers import Input,Dropout,Flatten,Conv2D,Activation,MaxPooling2D,Dense
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint,Callback,EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array,load_img
from keras import applications

In [9]:
# since the data is already prepared for other runs, I am directly using those
train_data_dir = '/Users/ram/Desktop/DS-PRACTICE-PYTHON/DEEPNN/Cat_Vs_Dog/train_sub'
validation_data_dir = '/Users/ram/Desktop/DS-PRACTICE-PYTHON/DEEPNN/Cat_Vs_Dog/val_sub'
nb_train_samples = 2000
nb_validation_samples = 1100
epochs=50
batch_size=16
top_model_weights_path = 'Users/ram/Desktop/DS-PRACTICE-PYTHON/DEEPNN/Cat_Vs_Dog/bottleneck_fc_model.h5'

In [11]:
# Defining the funciton for saving the weights from VGG16 training

def save_vgg16_weights():
    datagen = ImageDataGenerator(rescale=1./255)
    
    # build the VGG16 network
    model_vgg16_base = applications.VGG16(include_top=False, weights='imagenet')
    
    train_generator = datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(150,150),
                    batch_size=batch_size,
                    class_mode=None,
                    shuffle=False)
    vgg16_features_train = model_vgg16_base.predict_generator(
                    train_generator,
                    nb_train_samples // batch_size)
    np.save(open('vgg16_features_train.npy','w'),vgg16_features_train)
    
    val_generator = datagen.flow_from_directory(
                    validation_data_dir,
                    target_size=(150,150),
                    batch_size=batch_size,
                    class_mode=None,
                    shuffle=False)
    vgg16_features_val = model_vgg16_base.predict_generator(
                    val_generator,
                    nb_train_samples // batch_size)
    np.save(open('vgg16_features_val.npy','w'),vgg16_features_val)

In [12]:
def train_vgg16_top_model():
    train_data = np.load(open('vgg16_features_train.npy'))
    train_labels = np.array([0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))
    
    validation_data = np.load(open('vgg16_features_val.npy'))
    val_labels = np.array([0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    model.fit(train_data,train_lables,
             epochs=epochs,
             batch_size=batch_size,
             validation_data=(validation_data,validation_lables))
    model.save_weights(top_model_weights_path)

In [17]:
save_vgg16_weights()
train_vgg16_top_model()